# Critiquing survey questions
This notebook shows some ways of using EDSL to critique and improve survey questions. We do this by parameterizing a series of free text questions prompting an AI agent to provide feedback and criticism a survey question. We then ask the agent to provide a better version of the survey question, with and without the agent's responses to the prompts for feedback and criticism. We also compare results from different personas assigned to the agents and results using different LLMs.

In [1]:
# ! pip install edsl

In [2]:
from edsl.questions import QuestionFreeText
from edsl import Scenario, Survey, Agent, Model

In [3]:
q1 = QuestionFreeText(
    question_name = "problems",
    question_text = "What are some problems with this survey question: {{ draft_question }}"
)

q2 = QuestionFreeText(
    question_name = "confusing",
    question_text = "What are some ways in which this survey question may be confusing: {{ draft_question }}"
)

q3 = QuestionFreeText(
    question_name = "truthful",
    question_text = "What are some ways of ensuring that respondents will answer this survey question truthfully: {{ draft_question }}"
)

q4 = QuestionFreeText(
    question_name = "revised1",
    question_text = "Please provide an improved version of the following survey question: {{ draft_question }}"
)

# This question also prompts the agent to provide an improved version of the draft survey question, 
# but we will add the context of responses 1-3 to the prompt (see .add_targeted_memory() step below)
q5 = QuestionFreeText(
    question_name = "revised2",
    question_text = "Please provide an improved version of the following survey question: {{ draft_question }}"
)

draft_questions = [
    "Where are you from?",
    "What is your annual income?"
]

scenarios = [Scenario({"draft_question":q}) for q in draft_questions]

personas = [
    "", # No persona
    "You have some experience in responding to surveys.",
    "You are an expert in survey design and cognitive testing.",
]

agents = [Agent(traits={"persona":p}) for p in personas]

survey = Survey(questions = [q1, q2, q3, q4, q5])

# Here we add the context of responses 1-3 to the prompt for q5:
survey.add_targeted_memory(q5, q1)
survey.add_targeted_memory(q5, q2)
survey.add_targeted_memory(q5, q3)

results = survey.by(scenarios).by(agents).run(progress_bar=True)

Output()

In [4]:
results.columns

['agent.agent_name',
 'agent.persona',
 'answer.confusing',
 'answer.problems',
 'answer.revised1',
 'answer.revised2',
 'answer.truthful',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.confusing_system_prompt',
 'prompt.confusing_user_prompt',
 'prompt.problems_system_prompt',
 'prompt.problems_user_prompt',
 'prompt.revised1_system_prompt',
 'prompt.revised1_user_prompt',
 'prompt.revised2_system_prompt',
 'prompt.revised2_user_prompt',
 'prompt.truthful_system_prompt',
 'prompt.truthful_user_prompt',
 'raw_model_response.confusing_raw_model_response',
 'raw_model_response.problems_raw_model_response',
 'raw_model_response.revised1_raw_model_response',
 'raw_model_response.revised2_raw_model_response',
 'raw_model_response.truthful_raw_model_response',
 'scenario.draft_question']

In [5]:
results.select("persona", "problems", "confusing", "truthful", "revised1", "revised2").print()

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ agent            ┃ answer           ┃ answer           ┃ answer           ┃ answer           ┃ answer           ┃
┃ .persona         ┃ .problems        ┃ .confusing       ┃ .truthful        ┃ .revised1        ┃ .revised2        ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│                  │ The question     │ The survey       │ To increase the  │ Could you please │ What is your     │
│                  │ 'Where are you   │ question 'Where  │ likelihood of    │ specify your     │ current country  │
│                  │ from?' can be    │ are you from?'   │ respondents      │ country of       │ of residence?    │
│                  │ problematic due  │ can be confusing │ answering the    │ origin?          │                  │
│                  │ to its ambiguity │ for several      │ question 'Where  │                  │                  │
│                  │ and potential    │ reasons.         │ are you from?'   │                  │                  │
│                  │ for              │ Firstly, it can  │ truthfully, you  │                  │                  │
│                  │ misinterpretati… │ be unclear       │ can implement a  │                  │                  │
│                  │ Respondents      │ whether the      │ few strategies.  │                  │                  │
│                  │ might be unsure  │ question refers  │ Firstly, assure  │                  │                  │
│                  │ whether to       │ to one's place   │ anonymity and    │                  │                  │
│                  │ answer with      │ of birth,        │ confidentiality  │                  │                  │
│                  │ their current    │ current          │ to reduce social │                  │                  │
│                  │ residence,       │ residence, or    │ desirability     │                  │                  │
│                  │ hometown,        │ the location     │ bias. Secondly,  │                  │                  │
│                  │ country of       │ where one was    │ explain the      │                  │                  │
│                  │ origin, or       │ raised.          │ purpose of the   │                  │                  │
│                  │ ethnic           │ Secondly, for    │ question and how │                  │                  │
│                  │ background.      │ individuals who  │ the information  │                  │                  │
│                  │ Additionally, it │ have lived in    │ will be used,    │                  │                  │
│                  │ may be sensitive │ multiple places  │ which can        │                  │                  │
│                  │ for individuals  │ or come from     │ increase the     │                  │                  │
│                  │ who prefer not   │ multicultural    │ perceived        │                  │                  │
│                  │ to disclose      │ backgrounds, it  │ importance of    │                  │                  │
│                  │ their origin or  │ may be difficult │ providing an     │                  │                  │
│                  │ for those who    │ to pinpoint a    │ honest answer.   │                  │                  │
│                  │ have complex     │ single place     │ Additionally,    │                  │                  │
│                  │ identities. The  │ they identify    │ you might design │                  │                  │
│                  │ question also    │ with as 'from.'  │ the survey to be │                  │                  │
│                  │ assumes that     │ Additionally,    │ engaging and     │                  │                  │
│                  │ 'from' refers to │ the question may